# evaluation
ragas ref:


https://docs.ragas.io/en/stable/index.html

In [1]:
import os
from dotenv import load_dotenv
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from datasets import Dataset

from ragas.integrations.llama_index import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
import pandas as pd


# Load environment variables
load_dotenv()  # Load environment variables from .env file

# Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Initialize settings
Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")

from src_rag import ConfigManager, LoggerManager, Retriever, Reranker, ResponseSynthesizer, RAGQueryEngine, RAGASEvaluator
from llama_index.core import StorageContext, load_index_from_storage

import pandas as pd
from IPython.display import Markdown, display

import logging
logging.getLogger("httpx").setLevel(logging.WARNING)


%load_ext autoreload
%autoreload 2



/Users/juan/Documents/proyectos_propios/jobs/teamstation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup configuration and logging
config = ConfigManager()
api_key = config.load_config()

LoggerManager.setup_logging()

# Rebuild storage context and load the index
storage_context = StorageContext.from_defaults(persist_dir="./persist")
index = load_index_from_storage(storage_context)

# Initialize components
retriever = Retriever(index)
reranker = Reranker(api_key)
synthesizer = ResponseSynthesizer()

# Define parameters for both query engines
vector_top_k = 30
reranker_top_n = 5

# Instantiate the query engine with reranking
query_engine_with_reranker = RAGQueryEngine(
    retriever=retriever,
    reranker=reranker,
    synthesizer=synthesizer,
    vector_top_k=vector_top_k,
    reranker_top_n=reranker_top_n,
    with_reranker=True
)

# Instantiate the query engine without reranking
query_engine_without_reranker = RAGQueryEngine(
    retriever=retriever,
    reranker=reranker,  # Reranker is still passed, but reranking won't be used
    synthesizer=synthesizer,
    vector_top_k=vector_top_k,
    reranker_top_n=reranker_top_n,  # This parameter won't be used as reranking is disabled
    with_reranker=False
)

# Define your query
query = "What are potential CONSEQUENCES of Keytruda?"

# Query and display results using the engine with reranking
response_with_reranker = query_engine_with_reranker.query(query)
response_text_with_reranker = response_with_reranker.response
display(Markdown(f"**Response with Reranker:**\n{response_text_with_reranker}"))

# Query and display results using the engine without reranking
response_without_reranker = query_engine_without_reranker.query(query)
response_text_without_reranker = response_without_reranker.response
display(Markdown(f"**Response without Reranker:**\n{response_text_without_reranker}"))


INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
After Reranking, new rank list for nodes: [0, 5, 1, 3, 2, 4, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 27, 28, 11, 12, 13, 23, 26, 29]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response with Reranker:**
Potential consequences of Keytruda include immune-related adverse effects such as colitis, hepatitis, pneumonitis, and the exacerbation of pre-existing autoimmune diseases. It can also lead to symptoms like cough, shortness of breath, chest pain, and changes in vision that should be reported immediately for optimal care and safety.

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response without Reranker:**
Potential consequences of Keytruda include immune-related adverse effects such as colitis, hepatitis, pneumonitis, and changes in blood pressure. Additionally, Keytruda can lead to interactions with other medications like steroids and immunosuppressants, affecting its efficacy and safety. Close monitoring for symptoms like cough, chest pain, changes in vision, and neurological symptoms is essential while on Keytruda treatment.

# Ragas evaluation for RAG  systems

In [3]:

metrics = [
    faithfulness,
    answer_relevancy,
]

In [4]:
df = pd.read_parquet("../data/processed_data/df_eval.parquet")
ds = df[["augmented_questions"]].rename(columns = {"augmented_questions":"question"})
ds_dict = ds.to_dict()
ds_dict["question"] = [v for k, v in ds_dict["question"].items()] 

ds_dict = ds.to_dict()
ds_dict["question"] = [v for k, v in ds_dict["question"].items()] 

dataset = Dataset.from_dict(ds_dict)
dataset


Dataset({
    features: ['question'],
    num_rows: 220
})

In [5]:

# Initialize the evaluator
evaluator_llm = OpenAI(model="gpt-3.5-turbo")
evaluator = RAGASEvaluator(evaluator_llm)



In [6]:


result = evaluate(
    query_engine=query_engine_with_reranker,
    metrics=metrics,
    dataset=dataset,
    llm=evaluator_llm,
    embeddings=OpenAIEmbedding(),
)

df_result = result.to_pandas()
#df_result.to_parquet("../data/processed_data/df_result_ragas_without_reranking.parquet")

df_result

Running Query Engine:   0%|          | 0/220 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.open

Running Query Engine:   0%|          | 1/220 [00:02<10:35,  2.90s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Running Query Engine:   1%|▏         | 3/220 [00:03<02:53,  1.25it/s]

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:ragas.executor:Exception raised in Job[208]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-W3GgyascxGPcbC3NSuuLUYGT on tokens per min (TPM): Limit 200000, Used 197359, Requested 3137. Please try again in 148ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[208]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-W3GgyascxGPcbC3NSuuLUYGT on tokens per min (TPM): Limit 200000, Used 197359, Requested 3137. Please try again in 148ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'ra

Running Query Engine: 100%|█████████▉| 219/220 [01:32<00:00,  1.90it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
After Reranking, new rank list for nodes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Running Query Engine: 100%|██████████| 220/220 [01:35<00:00,  2.31it/s]


AttributeError: 'QueryResponse' object has no attribute 'source_nodes'

In [7]:
result

NameError: name 'result' is not defined

# Ragas

In [12]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
#from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    #context_precision,
    #context_recall,
    #harmfulness,
]

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# using GPT 3.5, use GPT 4 / 4-turbo for better accuracy
evaluator_llm = OpenAI(model="gpt-3.5-turbo")

df = pd.read_parquet("../data/processed_data/df_eval.parquet")


In [14]:
ds = df[["augmented_questions"]].rename(columns = {"augmented_questions":"question"})

ds_dict = ds.to_dict()
ds_dict["question"] = [v for k, v in ds_dict["question"].items()] 
ds_dict["question"]

['What are the frequently encountered adverse reactions associated with Keytruda?',
 'Which side effects are typically seen in patients receiving Keytruda therapy?']

In [15]:

ds_dict = ds.to_dict()
ds_dict["question"] = [v for k, v in ds_dict["question"].items()] 

dataset = Dataset.from_dict(ds_dict)
dataset

Dataset({
    features: ['question'],
    num_rows: 2
})

In [13]:

result = evaluate(
    query_engine=query_engine_with_reranker,
    metrics=metrics,
    dataset=dataset,
    llm=evaluator_llm,
    embeddings=OpenAIEmbedding(),
)

df_result = result.to_pandas()


df_result

AttributeError: 'RAGQueryEngine' object has no attribute 'aquery'

In [57]:
# without reranking
df_result.describe()

,faithfulness,answer_relevancy
count,218.000000,220.000000
mean,0.831684,0.859019
std,0.297590,0.301806
min,0.000000,0.000000
25%,0.666667,0.949170
50%,1.000000,0.964384
75%,1.000000,0.974852
max,1.000000,1.000000


In [45]:
# with reranking
df_result.describe()

,faithfulness,answer_relevancy
count,219.000000,220.000000
mean,0.793747,0.890620
std,0.314552,0.259305
min,0.000000,0.000000
25%,0.500000,0.949299
50%,1.000000,0.966063
75%,1.000000,0.978517
max,1.000000,1.000000
